In [1]:
import warnings                            # This block prevents display of harmless warnings, but should be
warnings.filterwarnings('ignore')          # commented out till the final version, to avoid missing "real" warnings

import kilroy_was_here                        # Mandatory. Allow access to shared python code from repository root
from audace.jupytools import iprint           # timstamped (to the ms) print with CPU and RAM consumption information
from audace.audiodataset import AudioDataset  # Class for audio dataset handling
from audace import providers
from audace import transformers
from audace import featurizers


# Path where to find initial annotated dataset (audio and lab files)
SOURCE_PATH ='/Users/jpg/Documents/Nolasco'

# Dataset name is the master key for dataset adressing. Change it according to the
# dataset you wan to generate
DATASET_NAME = 'DUO1000'

# Initialize Dataset Object. 
try:
    #By providing a source path,we implicitly indicates that you want to CREATE the data set.
    # Run with a pool of 4 processes
    iprint("Dataset creation. This may take up to 10 mn. Go grab a coffee.")
    ds = AudioDataset(DATASET_NAME, SOURCE_PATH, nprocs=4)
    
except FileExistsError:
    # To allow rerun, we catch the exception in case the dataset was already created.
    # Ideally, you should create the dataset once for all in a dedicated notebook,
    # and then retrieve it from other notebooks when needed
    # Here, by not providing a source path, we implicitly express the intent of RETRIEVING
    # an existing dataset rather than CREATING a new one
    iprint("Failing back to existing dataset retrieval")
    ds = AudioDataset(DATASET_NAME)
    iprint("Dataset retrieved")
    
# The following line provides some information about the newly created (or retrived) AudioDataset object    
ds.info()

ds.addLabel("nobee")
ds.addLabel("queen")

# The "queen" label value is deduced from the source file name, using a StringMapper transformer
# This transformer iterates over a list 2-uples (regular expression, target value) and return
# the target value as soon as a match is found. Thus, you must order your list from stricter to looser
trsfrm_queen = transformers.StringMapper(
        [('(?i)active', 1), 
         ('(?i)missing queen', 0),
         ('NO_QueenBee', 0),
         ('QueenBee', 1)     
        ])

# The transformer is then used over the source filenames, using the FromFileName labelizer
# This labelizer does not provide label strength.

n = ds.setLabel('queen', providers.FromFileName(trsfrm_queen))
iprint(n, "samples where processed for 'queen' label")

# The "nobee" label value comes from annotation files, (.lab files using the same base name as the audio
# source file they annotate), using the FromAnnotation labelizer, with no transformation.
# This labelizer takes 2 arguments:
# - a mandatory source path, pointing to the directory where the .lab files reside
# - an optional threshold, allowing to disregard any "label" event with a duration under this treshold
# The label strength over a sample is computed by summing the duration of "label" events (if > th) and dividing
# this sum by the sample duration.

# Here we use a 0.0s threshold
n = ds.setLabel('nobee', providers.FromAnnotation(SOURCE_PATH, th=0.0))
iprint(n, "samples where processed for 'nobee' label")

#The string matcher transformer behave differently than the StringMapper. It uses regexp
# capture group to retrieve part pf a string matching a specific pattern. This can be used
# either for complex or very basic matching. Here we just ask for the five first chars,
# provided they belong to characters valid for identifiers (A-Z, a-z,0-9 and underscore)
ds.addAttribute('hive')
ds.setAttribute('hive', providers.FromFileName(transformers.StringMatcher("^(\w{5})")))

#Here we use an MFCCfeaturiser, the providers beeing FromSample, which use audio chunks as a source
ds.addFeature('mfcc20')
ds.setFeature('mfcc20', providers.FromSample(ds.samples_path, featurizers.MFCC(n_mfcc=20)))




# Display dataset as a pandas dataframe
ds.dumpDataFrame()

[2020-08-13/12:42:49.925|14.1%|73.3%|0.26GB] Dataset creation. This may take up to 10 mn. Go grab a coffee.
[2020-08-13/12:42:49.925|00.0%|73.3%|0.26GB] >>>>> Starting Dataset DUO1000 build
[2020-08-13/12:42:49.957|41.2%|73.3%|0.26GB] Starting to process 4 audio files.
[2020-08-13/12:43:29.586|46.3%|64.7%|0.26GB] Creating Database
[2020-08-13/12:43:29.586|00.0%|64.7%|0.26GB] Database created
[2020-08-13/12:43:29.602|50.0%|64.7%|0.26GB] Please wait, computing checksum...
[2020-08-13/12:43:30.087|17.7%|65.7%|0.26GB]   Computed checksum e1de6cad8d2f2f15ec2df61c518af462
[2020-08-13/12:43:30.087|00.0%|65.7%|0.26GB]   Expected checksum d6e8796d8e357646cd0c6b0c4b3966a1
[2020-08-13/12:43:30.087|00.0%|65.7%|0.26GB] >>>>> Dataset DUO1000 successfully created.
[2020-08-13/12:43:30.087|00.0%|65.7%|0.26GB] ------------------------------------------------------
[2020-08-13/12:43:30.087|00.0%|65.7%|0.26GB] DATASET NAME          : DUO1000
[2020-08-13/12:43:30.087|00.0%|65.7%|0.26GB] DATASET PATH      


[2020-08-13/12:43:30.731|17.6%|65.8%|0.26GB] 2374 samples where processed for 'nobee' label


,name,file_id,start_t,end_t,nobee,queen,hive,mfcc20
0,00-000000,1,0.0,1.0,0.0,1.0,Hive1,"[-408.8679504394531, -408.73077392578125, -409..."
1,00-000001,1,1.0,2.0,0.0,1.0,Hive1,"[-423.73028564453125, -417.4234619140625, -409..."
2,00-000002,1,2.0,3.0,0.0,1.0,Hive1,"[-402.0168762207031, -405.77630615234375, -413..."
3,00-000003,1,3.0,4.0,0.0,1.0,Hive1,"[-415.6734619140625, -411.8334655761719, -410...."
4,00-000004,1,4.0,5.0,0.0,1.0,Hive1,"[-410.3650817871094, -402.5377197265625, -407...."
...,...,...,...,...,...,...,...,...
2369,03-000587,4,587.0,588.0,0.0,1.0,Hive3,"[-435.4527893066406, -431.11077880859375, -440..."
2370,03-000588,4,588.0,589.0,0.0,1.0,Hive3,"[-462.9355163574219, -454.59075927734375, -456..."
2371,03-000589,4,589.0,590.0,0.0,1.0,Hive3,"[-456.2651672363281, -455.3929138183594, -456...."
2372,03-000590,4,590.0,591.0,0.0,1.0,Hive3,"[-444.8450927734375, -443.4344787597656, -448...."
